In [1]:
import sys

# sys.path.append('/home/podcastle/workspace/vc-training-pipeline/')

from vocoder.models import Generator
from vocoder.utils import load_checkpoint

ModuleNotFoundError: No module named 'vocoder'

In [1]:
import torch
import numpy as np
import IPython.display as ipd
from omegaconf import OmegaConf
import matplotlib.pyplot as plt

from data.text import text_to_sequence
from train import load_model
from dataset import TextMelDurLoader

In [2]:
config_path = "mellotron-singer/configs/train.yaml"
synthesizer_checkpoint_path = "/home/podcastle/Documents/mellotron-singer/models/base_model/checkpoint_45000"
vocoder_checkpoint_path = "/home/podcastle/workspace/yeva_preprocessed/vocoder/checkpoints/g_02570000"

hparams_ = OmegaConf.load(config_path)
device = torch.device('cpu')

In [3]:
synthesizer = load_model(hparams_)
synthesizer.eval()
synthesizer.to(device)
print("Loading checkpoint '{}'".format(synthesizer_checkpoint_path))
synthesizer_checkpoint_dict = torch.load(synthesizer_checkpoint_path)
synthesizer.load_state_dict(synthesizer_checkpoint_dict['state_dict'])

769 1024 2
Loading checkpoint '/home/podcastle/Documents/mellotron-singer/models/base_model/checkpoint_45000'


<All keys matched successfully>

In [4]:
hparams = OmegaConf.load('/home/podcastle/workspace/vc-training-pipeline/configs/config.yaml')
vocoder = Generator(hparams).to(device)

vocoder_checkpoint_dict = load_checkpoint(vocoder_checkpoint_path, device)
vocoder.load_state_dict(vocoder_checkpoint_dict['generator'])
vocoder.eval()
vocoder.remove_weight_norm()

NameError: name 'Generator' is not defined

In [5]:
text = "Podcastle builds next generation audio content editing, and discovery tool for storytellers around the globe."
text = "In some amphibious aircrafts, the single engine is mounted on a pylon attached to the fuselage, which in turn is used as a floating hull."
# text = "Machine Learning has advanced many industries, leading to task automation, better ability to predict outcomes, and deep learning applications to interpret and generate text."

embed = np.load("/home/podcastle/workspace/yeva_preprocessed/YV.npy")[None, :]
sequence = text_to_sequence(text, ['english_cleaners'])
sequence = np.array(sequence)[None, :]

sequence = torch.from_numpy(sequence)
embed = torch.from_numpy(embed)

In [6]:
from mellotron_utils import get_data_from_musicxml


data = get_data_from_musicxml('data/haendel_hallelujah.musicxml', 132, convert_stress=True)
panning = {'Soprano': [-60, -30], 'Alto': [-40, -10], 'Tenor': [30, 60], 'Bass': [10, 40]}

FileNotFoundError: [Errno 2] No such file or directory: 'data/cmu_dictionary'

In [ ]:
_speakers_per_part = 4
frequency_scaling = 0.4
n_seconds = 90
audio_stereo = np.zeros((hparams.sampling_rate*n_seconds, 2), dtype=np.float32)
for i, (part, v) in enumerate(data.items()):
    rhythm = data[part]['rhythm'].cuda()
    pitch_contour = data[part]['pitch_contour'].cuda()
    text_encoded = data[part]['text_encoded'].cuda()
    
    for k in range(n_speakers_per_part):
        pan = np.random.randint(panning[part][0], panning[part][1])
        if any(x in part.lower() for x in ('soprano', 'alto', 'female')):
            speaker_id = torch.LongTensor([next(female_speakers)]).cuda()
        else:
            speaker_id = torch.LongTensor([next(male_speakers)]).cuda()
        print("{} MellotronID {} pan {}".format(part, speaker_id.item(), pan))

        with torch.no_grad():
            mel_outputs, mel_outputs_postnet, gate_outputs, alignments_transfer = mellotron.inference_noattention(
                (text_encoded, mel, speaker_id, pitch_contour*frequency_scaling, rhythm))

#             audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[0, 0]
#             audio = audio.cpu().numpy()
#             audio = panner(audio, pan)
#             audio_stereo[:audio.shape[0]] += audio 
            with torch.no_grad():
                y_g_hat = generator(mel_outputs_postnet.cpu())
                audio = y_g_hat.squeeze()
                audio = audio  * MAX_WAV_VALUE
                audio = audio.cpu().numpy().astype('int16')
                audio = panner(audio, pan)
                audio_stereo[:audio.shape[0]] += audio 

            write("{} {}.wav".format(part, speaker_id.item()), hparams.sampling_rate, audio)